# Interroger l'AIP Entrez du NCBI à l'aide de Biopython

### Quelques références pour l'API Entrez:

- [The 9 E-utilities and Associated Parameters](https://dataguide.nlm.nih.gov/eutilities/utilities.html)
- [A General Introduction to the E-utilities](https://www.ncbi.nlm.nih.gov/books/NBK25497/)
- [The E-utilities In-Depth: Parameters, Syntax and More](https://www.ncbi.nlm.nih.gov/books/NBK25499/)

Liste des bases de données disponibles [ici](https://www.ncbi.nlm.nih.gov/books/NBK25497/table/chapter2.T._entrez_unique_identifiers_ui/?report=objectonly).

### Références Biopython:

- [Guide d'utilisation au format pdf](http://biopython.org/DIST/docs/tutorial/Tutorial.pdf)
- [Site web](https://biopython.org/)

In [ ]:
# Attention. Biopython n'est pas une librairie standard de python; il faut d'abord l'installer !!
!pip install biopython # Ou avec Anaconda: 'conda install -c conda-forge biopython'

In [ ]:
from Bio import Entrez, SeqIO
Entrez.email = 'lemieux.mathieu@courrier.uqam.ca' # Remplacez par votre addresse courriel !!

### ESearch : Chercher dans une base de données à l'aide d'un mot-clé.

In [ ]:
def eSearch(db, term):
    handle = Entrez.esearch(db=db, term=term, retmax='5', sort='relevance')
    record = Entrez.read(handle)
    handle.close()
    return record

In [ ]:
kw     = input('Mot-clé: ')
db     = 'gene'
term   = f'human[organism] {kw}*[gene]' # Exemple de champs qui peuvent être utilisés pour construire la requête.

record = eSearch(db=db, term=term)
IdList = record['IdList']

print(IdList)

### ESummary : Sommaire d'information sur une entrée spécifique d'une base de donnée

In [ ]:
def eSummary(db, ids):
    handle = Entrez.esummary(db=db, id=ids)
    record = Entrez.read(handle)
    handle.close()
    return record

In [ ]:
IdList = ['7076', '2056', '2052', '2057', '8288']  # <- Exemple de liste d'identifiant de gènes
ids    = ','.join(IdList)  # <- On peut passer une liste de ids, à condition de transformer en chaîne, avec des virgules.
db     = 'gene'

record    = eSummary(db=db, ids=ids)
Summaries = record['DocumentSummarySet']['DocumentSummary']

for i in range(len(IdList)):
    name        = Summaries[i]['Name']
    description = Summaries[i]['Description']
    summary     = Summaries[i]['Summary']
    organism    = Summaries[i]['Organism']['ScientificName']
    print('\n', name, organism, description, summary, sep='\n')

### ELink : Accéder aux références croisées dans les bases de données

Très utile! Permet par exemple d'obtenir le numéro d'accession d'une séquence nucléotidique à l'aide de l'identifiant du gêne.

In [ ]:
def eLink(dbFrom, db, id, linkname):
    handle = Entrez.elink(dbFrom=dbFrom, db=db, id=id, linkname=linkname)
    record = Entrez.read(handle)
    handle.close()
    return record

In [ ]:
dbFrom   = 'gene'
db       = 'nuccore'
linkname = 'gene_nuccore_refseqgene'
idGene   = '2056'  # <- Exemple d'identifiant de gène

record   = eLink(dbFrom=dbFrom, db=db, id=idGene, linkname=linkname)
idRefSeq = record[0]['LinkSetDb'][0]['Link'][0]['Id']

print(idRefSeq)

### EFetch : Accéder à des formats de fichier spécifiques (ex. fiche GenBank, fichier Fasta)

On utilise record = SeqIO.read() au lieu de record = Entrez.read()

In [ ]:
def eFetch(db, id, rettype, recordFormat):
    handle = Entrez.efetch(db=db, id=id, rettype=rettype)
    record = SeqIO.read(handle, recordFormat)
    handle.close()
    return record

#### Téléchargement de la fiche GenBank

In [ ]:
db           = 'nucleotide'
rettype      = 'gb'
recordFormat = 'genbank'
id           = '1243022497'  # <- Exemple de numéro d'accession
# id           = '1243022497'  # <- Exemple de référence croisée RefSeq

record  = eFetch(db=db, id=id, rettype=rettype, recordFormat=recordFormat)

# Affichage de tout le contenu
print(record, '\n')

# Affichage des 'Features'
for feature in record.features: print(feature)
    
# Affichage de la séquence
print(record.seq)

# Enregistrement de la fiche GenBank
SeqIO.write(record, 'nom_de_fichier.gb', 'gb')

#### Téléchargement du fichier Fasta

In [ ]:
db           = 'nucleotide'
rettype      = 'fasta'
recordFormat = 'fasta'
idRefSeq     = '1243022497'  # <- Exemple de référence croisée RefSeq

record  = eFetch(db=db, id=idRefSeq, rettype=rettype, recordFormat=recordFormat)

print(record)
record.description = record.id  # <- On peut modifier le descriptif du fasta au besoin.
                                #    Pratique avant de faire des arbres phylogénétiques sur des alignements multiples

# Enregistrement du fichier Fasta
SeqIO.write(record, 'nom_de_fichier.fasta', 'fasta')